In [3]:
import tensorflow as tf
import argparse
import yaml
from utils import read_dataset,read_test_dataset,TextIterator,add_extras,prepare_input_feed

[[0, 13, 14, 15, 4], [0, 21, 22, 23, 21, 24, 23, 21, 25, 23, 26, 4], [0, 16, 17, 18, 4], [0, 8, 9, 10, 4]] [[0, 6, 2], [0, 8, 2], [0, 7, 2], [0, 4, 2]]
[[0, 13, 19, 20, 4], [0, 11, 12, 11, 4], [0, 5, 6, 7, 4], [0, 1, 2, 3, 4]] [[0, 6, 2], [0, 5, 2], [0, 3, 2], [0, 1, 2]]


In [ ]:
'''parser = argparse.ArgumentParser(description='Train model')
parser.add_argument('--fname', type=str, default='fname', metavar='F',
                    help="image to be masked,output stored at name_mask.jpg")
args = parser.parse_args()
timestr = time.strftime("%Y%m%d%H%M%S")
log_file=log_dir+"//log"+timestr+".txt"
dtype = tf.float16 # else tf.float32

test_path=cfg['test_path']
emb_dim=cfg['emb_dim']
src_embed_path=cfg['src_embed_path']
target_embed_path=cfg['target_embed_path']
max_gradient_norm=cfg['max_gradient_norm']
min_learning_rate=cfg['min_learning_rate']
data_dir=cfg['data_dir']
source_train_path=cfg['source_train_path']
target_train_path=cfg['target_train_path']
source_dev_path=cfg['source_dev_path']
target_dev_path=cfg['target_dev_path']
ref_path=cfg['ref_path']
graph_path=cfg['graph_path']
checkpoint_dir=cfg['checkpoint_dir']
num_layers=cfg['num_layers']
train_dir=cfg['train_dir']
batch_size=cfg['batch_size']
learning_rate=cfg['learning_rate']
learning_rate_decay_factor=cfg['learning_rate_decay_factor']
max_train_data_size=cfg['max_train_data_size']
steps_per_checkpoint=cfg['steps_per_checkpoint']
num_epochs=cfg['num_epochs']
keep_prob=cfg['keep_prob']

embed_dir="embed"
checkpoint_path = os.path.join(checkpoint_dir, "translate.ckpt")'''

In [ ]:
def create_model(session, forward_only,src_embeddings=None,target_embeddings=None):
    #Use forward only for decode
    """Create translation model and initialize or load parameters in session."""
    model = LID_model()

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
        model.saver.restore(session, ckpt.model_checkpoint_path)
    else:
        print("Created model with fresh parameters.")
        session.run(tf.global_variables_initializer())
    return model

In [ ]:
def train():
    flog = open(log_file, 'a')
    x_dataset,v=read_dataset_x(x_train_path)
    y_dataset,classes=read_dataset(y_train_path)
    v=add_extras(v)
    n=len(x_dataset)
    n1=int(0.8*n)
    train_iterator=TextIterator(x_dataset[0:n1],y_dataset[0:n1],batch_size)
    val_iterator=TextIterator(x_dataset[n1:],y_dataset[n1:],batch_size)
    with tf.Session() as sess:
    # Create model.
        
        model = create_model(sess, False)

        # This is the training loop.
        valid_acc=[]
        max_acc=0
        for epoch in range(num_epochs):
            

            i=0
            for batch_X,batch_Y in train_iterator:
                
                start_time = time.time()
                #Prepare data inplaceholder format
                input_feed=data_utils.prepare_input_feed(model, batch_X, batch_Y,max_seq_len)

                output_feed = [model.update,  # Update Op that does SGD.
                              model.norm,  # Gradient norm.
                              model.loss]  # Loss for this batch.
                _,_,loss = sess.run(output_feed, input_feed)
                print("Loss at batch ",i,loss)
                
            acc=0
            n=0
            for batch_X,batch_Y in val_iterator:
                input_feed=data_utils.prepare_input_feed(model, batch_X, batch_Y,max_seq_len)
                output_feed=[model.accuracy]
                v=sess.run(output_feed, input_feed)
                acc+=v
                n+=1
            acc/=n
            print("Val acc at epoch ",epoch,acc)
            #If average epoch loss stays constant for 3 epochs decrease learning rate
            if len(valid_acc) > 2 and min(valid_acc[-3])>acc:
                sess.run(model.learning_rate_decay_op)
                lr=sess.run(model.learning_rate)
                if lr<min_learning_rate:
                    print("Reached convergence at epoch " + str(epoch) + "\n")
                    break
            valid_acc.append(acc)

            #If average loss over the epoch has reduced, save checkpoint
            if(acc > max_acc ):
                model.saver.save(sess, checkpoint_path, global_step=model.global_step)
                print("Saved checkpoint")
               

            #Store graph
            writer = tf.summary.FileWriter(graph_path, sess.graph)
            writer.close()
    